In [27]:
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")

For static replication of any constant maturity swap (CMS) payoff $g(F)$, where $F$ is the swap rate, we use the following formula:

  \begin{equation*}
    \begin{split}
      V_0 &= D(0,T) g(F) + h'(F)[V^{pay}(F)-V^{rec}(F)] \\
      &\;\;\;\;\;\;\;\;\;\;+ \int_0^F h''(K) V^{rec}(K) dK +
      \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

where

  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\text{IRR}(K)} \\
      h'(K) &= \frac{\text{IRR}(K)g'(K) - g(K)\text{IRR}'(K)}{\text{IRR}(K)^2} \\
      h''(K) &= \frac{\text{IRR}(K)g''(K)-\text{IRR}''(K)g(K) -2\cdot\text{IRR}'(K)g'(K)}{\text{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\text{IRR}'(K)^2g(K)}{\text{IRR}(K)^3}.
    \end{split}
  \end{equation*}

We can implement this in Python. First we define the IRR functions.

In [28]:
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative


For CMS rate payment, since $g(F)=F^\frac{1}{p} - 0.04^\frac{1}{q}, p = 4, q= 2$, we have the derivatives:

\begin{equation*}
\begin{split}
g(K) &= K^\frac{1}{4} - 0.04^\frac{1}{2} \\
g'(K) &= \frac{1}{4}K^{-\frac{3}{4}} \\
g''(K) &= -\frac{3}{16}K^{-\frac{7}{4}}
\end{split}
\end{equation*}

In [29]:
def g_0(K):
    return K**(1/4) - 0.04**(1/2)

def g_1(K):
    return (1/4) * K**(-3/4)

def g_2(K):
    return (-3/16) * K**(-7/4)


In [30]:
def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative

In [31]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

We will also need to implement the IRR-settled payer and receiver swaption formulae:

  \begin{equation*}
    \begin{split}
      V^{pay}_{n,N}(0) &= D(0,T_n) \cdot \text{IRR}(S_{n,N}(0)) \cdot \text{Black76Call}(S_{n,N}(0),K,\sigma_{n,N},T) \\
      V^{rec}_{n,N}(0) &= D(0,T_n) \cdot \text{IRR}(S_{n,N}(0)) \cdot \text{Black76Put}(S_{n,N}(0),K,\sigma_{n,N},T) \\
    \end{split}
  \end{equation*}

where $S_{n,N}(0)=F$ is today's forward swap rate calculated based on the curves we bootstrapped, and $\sigma_{n,N}$ is the SABR implied volatility calibrated to swaption market data.

In [32]:
def Black76Call(F, K, sigma, T):
    d1 = (np.log(F/K) + (sigma**2)*T/2)/(sigma*np.sqrt(T))
    d2 = (np.log(F/K) - (sigma**2)*T/2)/(sigma*np.sqrt(T))
    return (F*norm.cdf(d1) - K*norm.cdf(d2))

def Black76Put(F, K, sigma, T):
    d1 = (np.log(F/K) + (sigma**2)*T/2)/(sigma*np.sqrt(T))
    d2 = (np.log(F/K) - (sigma**2)*T/2)/(sigma*np.sqrt(T))
    return (K*norm.cdf(-d2) - F*norm.cdf(-d1))

In [33]:
def payer_swaption(discount, F, K, sigma, T, m , N):
    return discount * IRR_0(F, m, N) * Black76Call(F, K, sigma, T)

def receiver_swaption(discount, F, K, sigma, T, m , N):
    return discount * IRR_0(F, m, N) * Black76Put(F, K, sigma, T)

In [34]:
def call_integrand(discount, F, K, sigma, T, m , N):
    price = h_2(K, m, N) * payer_swaption(discount, F, K, sigma, T, m , N)
    return price

def put_integrand(discount, F, K, sigma, T, m , N):
    price = h_2(K, m, N) * receiver_swaption(discount, F, K, sigma, T, m , N)
    return price

# Part 1

For CMS rate payoff, the payoff function can be defined simply as $g(F)=F$, and the static replication formula simplifies into:

  \begin{equation*}
    \begin{split}
      D(0,T) F + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

In [35]:
# Input parameters here
discount = 0.982184     #OIS discount factor for D0(0, 5y) from part 1
F = 0.043634            #Forward swap rate for 5y x 10y swaption from part 1
T = 5                   #Option expiry date
N = 10                  #Tenor of swap
m = 2

alpha = 0.170869        #Used calibrated parameters from part 2
beta = 0.9
rho = -0.372221
nu=0.532554

In [36]:
# sigma = SABR(F, F, T, alpha=alpha, beta=beta, rho=rho, nu=nu)
I_put = quad(lambda x: put_integrand(discount, F, x, SABR(F,x,T, alpha=alpha, beta=beta, rho=rho, nu=nu), T, m , N), 0.0, F)
I_call = quad(lambda x: call_integrand(discount, F, x, SABR(F,x,T, alpha=alpha, beta=beta, rho=rho, nu=nu), T, m , N), F, 5000)

v_0 = discount * g_0(F) + I_put[0] + I_call[0]

print(f"The PV of the payoff of the decompounded option in part 1 is: {v_0}")

The PV of the payoff of the decompounded option in part 1 is: 0.23363627577761975


# Part 2

<div class="alert alert-block alert-warning">
<b>To Check:</b> Check how to set the limit and where to use the new strike
</div>

If payoff is now $(\text {CMS 10y}^\frac{1}{p} - 0.04^\frac{1}{q})^+$, then the payoff can be replicated with 

  \begin{equation*}
    \begin{split}
      V_0 &= h'(L)V^{pay}(L) + \int_L^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

For CMS rate payoff, the payoff function can be defined simply as $g(F)=F$. 

  \begin{equation*}
      \begin{split}
      F^\frac{1}{4} > 0.2 \\
      F > 0.2^4 = L
      \end{split}
  \end{equation*}

In [39]:
L = (0.04** (1/2)) ** 4
# sigma_2 = SABR(F, L, T, alpha=alpha, beta=beta, rho=rho, nu=nu)
I_call2 = quad(lambda x: call_integrand(discount, F, x, SABR(F,x,T, alpha=alpha, beta=beta, rho=rho, nu=nu), T, m , N), L, 5000)
v_1 = h_1(L, m, N) * payer_swaption(discount, F, L, SABR(F,L,T, alpha=alpha, beta=beta, rho=rho, nu=nu), T, m, N) + I_call2[0]

In [40]:
print(f"The PV of the payoff of the decompounded option in part 2 is: {v_1}")

The PV of the payoff of the decompounded option in part 2 is: 0.25196852191752084
